In [60]:
#Haris Amaldi
#Dilarang mengcopy code. PLAGIARISME ITU HARAM!!!!

In [61]:
#Membuka file dataset
!unzip datafilm.zip

Archive:  datafilm.zip
replace movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: an
error:  invalid response [an]
replace movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [62]:
#Mengimport fungsi-fungsi yang dibutuhkan dan mencari banyaknya data film dan pengguna yang tercatat.
import os
import plotly.express as px
import numpy as np
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
import re
from sklearn.neighbors import NearestNeighbors
import random
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

film = pd.read_csv("movies.csv")
rating = pd.read_csv("ratings.csv")

print('Jumlah data film: ', len(film.movieId.unique()))
print('Jumlah data pengguna: ', len(rating.userId.unique()))

Jumlah data film:  9742
Jumlah data pengguna:  610


In [63]:
#Melihat bentuk data film
film.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [64]:
#Melihat bentuk data rating
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [65]:
# Menggabungkan seluruh movieId pada kategori film dan rating
film_all = np.concatenate((
    film.movieId.unique(),
    rating.movieId.unique()
))
 
# Mengurutkan data dan menghapus data yang sama
film_all = np.sort(np.unique(film_all))
 
print('Jumlah seluruh data film berdasarkan movieID: ', len(film_all))

Jumlah seluruh data film berdasarkan movieID:  9742


In [66]:
filmrating = pd.merge(film, rating , on='movieId', how='left')
filmrating

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [67]:
print("Rating yang tersedia:", filmrating.rating.unique())

Rating yang tersedia: [4.  4.5 2.5 3.5 3.  5.  0.5 2.  1.5 1.  nan]


In [68]:
filmrating.isnull().sum()

movieId       0
title         0
genres        0
userId       18
rating       18
timestamp    18
dtype: int64

In [69]:
filmrating = filmrating.dropna()
filmrating

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [70]:
# Memisahkan tahun rilis film menjadi 1 kolom sendiri
filmrating['tahunrilis'] = filmrating.title.str.extract('.*\((.*)\).*')
filmrating.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,movieId,title,genres,userId,rating,timestamp,tahunrilis
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,1995
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,1995
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,1995
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,1995
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,1995


In [71]:
# Menghapus tahun dari kolom judul
filmrating['title'] = filmrating.title.str.split('(').str[0].str[:-1]
filmrating.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,movieId,title,genres,userId,rating,timestamp,tahunrilis
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,1995
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,1995
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,1995
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,1995
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,1995


In [72]:
#mengubah timestamp menjadi data yang bisa dibaca
def UNIX_to_Readable(df):
    return pd.to_datetime(datetime.fromtimestamp(df).strftime('%Y-%m-%d %H:%M:%S'))


filmrating.timestamp = filmrating.timestamp.apply(UNIX_to_Readable)
filmrating.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,movieId,title,genres,userId,rating,timestamp,tahunrilis
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,2000-07-30 18:45:03,1995
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,1996-11-08 06:36:02,1995
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,2005-01-25 06:52:26,1995
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,2017-11-13 12:59:30,1995
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,2011-05-18 05:28:03,1995


In [73]:
# Menghapus nilai rating berkoma (Tidak bulat)
filmrating.rating = np.ceil(filmrating.rating)
print("Rating yang tersedia:", filmrating.rating.unique())

Rating yang tersedia: [4. 5. 3. 1. 2.]


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [74]:
#Pengujian penambahan Jumlah rating agar ketahuan apakah ada yang hanya dirating kurang dari 10 pengguna.
filmrating.groupby('movieId').sum()

,userId,rating
movieId,,
1,65904.0,866.0
2,36251.0,394.0
3,14747.0,174.0
4,1539.0,17.0
5,14679.0,154.0
...,...,...
193581,184.0,4.0
193583,184.0,4.0
193585,184.0,4.0


In [75]:
#Ada yang jumlah ratingnya dibawah rentang 10-50. Harus dihapus terlebih dahulu biar lebih cepat dan efektif.
movieFrequency_greater_10 = filmrating['movieId'].value_counts()[filmrating['movieId'].value_counts() >= 10].index
filmrating = filmrating[filmrating.movieId.isin(movieFrequency_greater_10)]

In [76]:
#Preparasi data
preparasi = filmrating
preparasi.sort_values('movieId')

,movieId,title,genres,userId,rating,timestamp,tahunrilis
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,2000-07-30 18:45:03,1995
137,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,382.0,5.0,2018-01-05 14:30:28,1995
138,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,385.0,4.0,1996-06-13 18:47:22,1995
139,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,389.0,5.0,1997-03-09 19:02:54,1995
140,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,391.0,3.0,2002-09-18 22:27:57,1995
...,...,...,...,...,...,...,...
100804,187593,Deadpool 2,Action|Comedy|Sci-Fi,62.0,4.0,2018-05-25 18:47:51,2018
100803,187593,Deadpool 2,Action|Comedy|Sci-Fi,25.0,5.0,2018-08-28 15:35:34,2018
100813,187593,Deadpool 2,Action|Comedy|Sci-Fi,514.0,4.0,2018-09-17 02:18:46,2018
100807,187593,Deadpool 2,Action|Comedy|Sci-Fi,248.0,5.0,2018-08-18 13:45:09,2018


In [77]:
preparasi = preparasi.drop_duplicates('movieId')
preparasi

,movieId,title,genres,userId,rating,timestamp,tahunrilis
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,2000-07-30 18:45:03,1995
215,2,Jumanji,Adventure|Children|Fantasy,6.0,4.0,1996-10-17 11:58:42,1995
325,3,Grumpier Old Men,Comedy|Romance,1.0,4.0,2000-07-30 18:20:47,1995
384,5,Father of the Bride Part II,Comedy,6.0,5.0,1996-10-17 12:05:38,1995
433,6,Heat,Action|Crime|Thriller,1.0,4.0,2000-07-30 18:37:04,1995
...,...,...,...,...,...,...,...
100526,174055,Dunkirk,Action|Drama|Thriller|War,50.0,3.0,2018-02-09 09:30:47,2017
100594,176371,Blade Runner 2049,Sci-Fi,25.0,4.0,2018-08-28 15:34:29,2017
100641,177765,Coco,Adventure|Animation|Children,89.0,2.0,2018-03-07 07:38:02,2017
100688,179819,Star Wars: The Last Jedi,Action|Adventure|Fantasy|Sci-Fi,62.0,4.0,2018-04-06 21:03:25,2017


In [78]:
Idfilm = preparasi['movieId'].tolist()
Judul = preparasi['title'].tolist()
Genre = preparasi['genres'].tolist()

print(len(Idfilm))
print(len(Judul))
print(len(Genre))

2269
2269
2269


In [79]:
#Membuat dictionary baru
dictbaru = pd.DataFrame({
    'idfilm': Idfilm,
    'Judulfilm': Judul,
    'Genrefilm': Genre
})
dictbaru

,idfilm,Judulfilm,Genrefilm
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,5,Father of the Bride Part II,Comedy
4,6,Heat,Action|Crime|Thriller
...,...,...,...
2264,174055,Dunkirk,Action|Drama|Thriller|War
2265,176371,Blade Runner 2049,Sci-Fi
2266,177765,Coco,Adventure|Animation|Children
2267,179819,Star Wars: The Last Jedi,Action|Adventure|Fantasy|Sci-Fi


Ini content based filtering

In [80]:
data = dictbaru
data.sample(5)

,idfilm,Judulfilm,Genrefilm
2003,69844,Harry Potter and the Half-Blood Prince,Adventure|Fantasy|Mystery|Romance|IMAX
1044,2924,Drunken Master,Action|Comedy
2243,136864,Batman v Superman: Dawn of Justice,Action|Adventure|Fantasy|Sci-Fi
1564,6440,Barton Fink,Drama|Thriller
1963,62999,Madagascar: Escape 2 Africa,Action|Adventure|Animation|Children|Comedy|IMAX


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()
 
# Melakukan perhitungan idf pada data genre film
tf.fit(data['Genrefilm']) 
 
# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names() 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'fi',
 'film',
 'horror',
 'imax',
 'musical',
 'mystery',
 'noir',
 'romance',
 'sci',
 'thriller',
 'war',
 'western']

In [82]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data['Genrefilm']) 
 
# Melihat ukuran matrix tfidf
tfidf_matrix.shape 

(2269, 21)

In [83]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0.        , 0.37515221, 0.54947369, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.47780142, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.44980472, 0.65881488, ..., 0.        , 0.        ,
         0.        ],
        [0.36542737, 0.41020987, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.44882101, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [84]:
# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan genre film
# Baris diisi dengan judul film
 
pd.DataFrame(
    tfidf_matrix.todense(), 
    columns=tf.get_feature_names(),
    index=data.Judulfilm
).sample(21, axis=1).sample(10, axis=0)

,comedy,drama,imax,animation,sci,romance,fantasy,children,noir,fi,...,film,adventure,war,action,documentary,thriller,horror,crime,mystery,musical
Judulfilm,,,,,,,,,,,,,,,,,,,,,
Deadpool,0.342600,0.000000,0.000000,0.0,0.509665,0.000000,0.0,0.0,0.0,0.509665,...,0.0,0.449943,0.0,0.400823,0.0,0.000000,0.0,0.0,0.0,0.0
9 1/2 Weeks,0.000000,0.571770,0.000000,0.0,0.000000,0.820414,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
8 Mile,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Stargate,0.000000,0.000000,0.000000,0.0,0.542497,0.000000,0.0,0.0,0.0,0.542497,...,0.0,0.478927,0.0,0.426643,0.0,0.000000,0.0,0.0,0.0,0.0
Broadcast News,0.507413,0.492696,0.000000,0.0,0.000000,0.706953,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Spider-Man 3,0.000000,0.000000,0.591203,0.0,0.399812,0.000000,0.0,0.0,0.0,0.399812,...,0.0,0.352962,0.0,0.314429,0.0,0.327614,0.0,0.0,0.0,0.0
GoldenEye,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.613704,0.0,0.546706,0.0,0.569631,0.0,0.0,0.0,0.0
Screamers,0.000000,0.000000,0.000000,0.0,0.551323,0.000000,0.0,0.0,0.0,0.551323,...,0.0,0.000000,0.0,0.433584,0.0,0.451765,0.0,0.0,0.0,0.0
Déjà Vu,0.000000,0.000000,0.000000,0.0,0.551323,0.000000,0.0,0.0,0.0,0.551323,...,0.0,0.000000,0.0,0.433584,0.0,0.451765,0.0,0.0,0.0,0.0


In [85]:
from sklearn.metrics.pairwise import cosine_similarity
 
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

array([[1.        , 0.78516345, 0.16656305, ..., 0.83403351, 0.39750186,
        0.10958345],
       [0.78516345, 1.        , 0.        , ..., 0.60118955, 0.50626638,
        0.        ],
       [0.16656305, 0.        , 1.        , ..., 0.        , 0.        ,
        0.22369144],
       ...,
       [0.83403351, 0.60118955, 0.        , ..., 1.        , 0.18451433,
        0.        ],
       [0.39750186, 0.50626638, 0.        , ..., 0.18451433, 1.        ,
        0.69437005],
       [0.10958345, 0.        , 0.22369144, ..., 0.        , 0.69437005,
        1.        ]])

In [86]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa judul film
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['Judulfilm'], columns=data['Judulfilm'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap judul film
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (2269, 2269)


Judulfilm,Bedtime Stories,"Grifters, The",Escape from L.A.,"Tuxedo, The","400 Blows, The"
Judulfilm,,,,,
"Fly, The",0.000000,0.065739,0.611701,0.000000,0.162953
Dave,0.241602,0.000000,0.000000,0.378861,0.000000
"Bronx Tale, A",0.000000,0.218046,0.000000,0.000000,0.540492
Lady and the Tramp,0.544296,0.000000,0.000000,0.208176,0.000000
American Wedding,0.414342,0.000000,0.000000,0.649738,0.000000
Starman,0.237759,0.070437,0.681901,0.000000,0.174600
Everything You Always Wanted to Know About Sex * But Were Afraid to Ask,0.414342,0.000000,0.000000,0.649738,0.000000
What Lies Beneath,0.000000,0.075170,0.000000,0.000000,0.186332
Crimson Tide,0.000000,0.080204,0.187578,0.000000,0.198810


In [87]:
def film_recommendations(nama_film, similarity_data=cosine_sim_df, items=data[['Judulfilm', 'Genrefilm']], k=5):
 
    index = similarity_data.loc[:,nama_film].to_numpy().argpartition(
        range(-1, -k, -1))
    
    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    # Drop nama_film agar nama film yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(nama_film, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

In [88]:
data[data.Judulfilm.eq('Toy Story')]

,idfilm,Judulfilm,Genrefilm
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy


In [89]:
film_recommendations('Toy Story')

,Judulfilm,Genrefilm
0,"Emperor's New Groove, The",Adventure|Animation|Children|Comedy|Fantasy
1,Moana,Adventure|Animation|Children|Comedy|Fantasy
2,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
3,Antz,Adventure|Animation|Children|Comedy|Fantasy
4,Toy Story 2,Adventure|Animation|Children|Comedy|Fantasy
